In [ ]:
%%time

In [9]:
# load urls
import json

with open('./data/urls.json') as f:
    urls = json.load(f)['urls']

In [10]:
urls

['https://www.notion.so/Atelier-nouveaux-usages-ff2ffa13eeb94d91a2d672d56b62be0a-5198447ed79746c590da4c3c9cb0cc81#cca152a983b148bbb4ef4323a4cbeb50',
 'https://www.notion.so/Atelier-nouveaux-usages-ff2ffa13eeb94d91a2d672d56b62be0a-5198447ed79746c590da4c3c9cb0cc81#3258f0238d2847a0a5d5c17215893ecc',
 'https://www.notion.so/Atelier-nouveaux-usages-ff2ffa13eeb94d91a2d672d56b62be0a-5198447ed79746c590da4c3c9cb0cc81#ff3f205c57e743bcbb1fd3b3f06abc4e',
 'https://www.notion.so/Atelier-nouveaux-usages-ff2ffa13eeb94d91a2d672d56b62be0a-5198447ed79746c590da4c3c9cb0cc81#f499d30971934fe19aeeaf26747f8f39',
 'https://www.notion.so/Atelier-nouveaux-usages-ff2ffa13eeb94d91a2d672d56b62be0a-5198447ed79746c590da4c3c9cb0cc81#6ce53a54a2bd4256afbaf077fbba9d1e',
 'https://www.notion.so/Atelier-nouveaux-usages-ff2ffa13eeb94d91a2d672d56b62be0a-5198447ed79746c590da4c3c9cb0cc81#7cd489356fb5478283124624cd5b9b4d',
 'https://www.notion.so/Atelier-nouveaux-usages-ff2ffa13eeb94d91a2d672d56b62be0a-5198447ed79746c590da4c3c9

In [11]:
from alive_progress import alive_bar
import time

with alive_bar(1000, force_tty=True) as bar:
    for i in range(1000):
        time.sleep(0.001)
        bar()

|████████████████████████████████████████| 1000/1000 [100%] in 1.2s (832.19/s)  


In [12]:
from dotenv import load_dotenv

load_dotenv("./.env")

True

In [13]:
import os
NOTION_TOKEN = os.environ['NOTION_TOKEN']

In [14]:
import polars as pl
import requests
from tqdm import tqdm
from alive_progress import alive_bar
import re

codes_naf = []
codes_sp = []
definitions = []
groupes = []
groupes_distinct = []


with alive_bar(len(urls), force_tty=True) as bar:
    for url in urls:
        block_id = url[112:]
        # print(block_id)
        url = f"https://api.notion.com/v1/blocks/{block_id}"

        headers = {
            "Accept": "application/json",
            "Notion-Version": "2022-02-22",
            "Authorization": NOTION_TOKEN
        }

        response = requests.get(url, headers=headers)
        text = response.text
        # nom_groupe est le nom que l'on voulait donner au groupement de code NAF (un par URL)
        nom_groupe = response.json()['numbered_list_item']['rich_text'][0]['text']['content']
        groupes_distinct.append(nom_groupe)

        # codes_list est la liste des codes naf récupérés sur la page
        codes_list = re.findall(r"\d\d\d\d[A-Z]", text)
        
        codes_list = codes_list[:(len(codes_list) // 2)]
        # definition est la définition donnée au code NAF correspondant
        definition = re.findall(r"\((.*?)\)", text)
        definition = definition[:(len(definition) // 2)]
        definition = definition[1:]

        definitions += definition

        for code in codes_list:
            codes_sp.append(code)
            code_naf = code[0:2] + '.' + code[2:]
            codes_naf.append(code_naf)
            groupes.append(nom_groupe)
        bar()

with open("output/interesting_naf_codes.csv", "w", encoding='utf-8') as file:
    file.write("Code NAF|Définition|Groupe \n")
    for i in range(len(codes_sp)):
        file.write(codes_sp[i])
        file.write('|')
        file.write(definitions[i])
        file.write('|')
        file.write(groupes[i])
        file.write('\n')


|████████████████████████████████████████| 15/15 [100%] in 8.9s (1.58/s)        


# now deal with big csvs

In [15]:
GEOLOC_PATH = r".\data\GeolocalisationEtablissement_Sirene_pour_etudes_statistiques_utf8\GeolocalisationEtablissement_Sirene_pour_etudes_statistiques_utf8.csv"

In [16]:
with open(GEOLOC_PATH) as f:
    print("headers : ", f.readline())
    print('first line : ', f.readline())

headers :  siret;x;y;qualite_xy;epsg;plg_qp24;plg_iris;plg_zus;plg_qp15;plg_qva;plg_code_commune;distance_precision;qualite_qp24;qualite_iris;qualite_zus;qualite_qp15;qualite_qva;y_latitude;x_longitude

first line :  30014695800081;824581.79;6621952.27;11;2154;CSZ;CSZ;CSZ;CSZ;CSZ;71201;;;;;;;46.686085838;4.6301694609



In [17]:
reader = pl.read_csv_batched(
    GEOLOC_PATH,
    separator=";",
    ignore_errors=True,
    columns=['siret', 'x_longitude', 'y_latitude', 'plg_code_commune']
)

In [18]:
first_dataframe = reader.next_batches(1)[0]

In [19]:
first_dataframe = first_dataframe.with_columns((first_dataframe["plg_code_commune"]//1000).rename("departement"))

In [20]:
first_dataframe

siret,plg_code_commune,y_latitude,x_longitude,departement
i64,i64,f64,f64,i64
30014695800081,71201,46.686086,4.630169,71
30462262400027,66114,42.60408,2.939352,66
31025238200026,73023,45.226824,6.739652,73
31163345700024,16368,45.597482,0.331143,16
3732059500079,6025,43.809367,7.187481,6
…,…,…,…,…
30779166500010,92050,48.91264,2.225717,92
30307287000015,92050,48.910815,2.221651,92
30449598900010,92050,48.904439,2.209469,92


In [21]:
departements = [62, 59]

In [22]:
first_dataframe.filter(pl.col("departement").is_in(departements))

siret,plg_code_commune,y_latitude,x_longitude,departement
i64,i64,f64,f64,i64
30989139800039,59636,50.69481,3.028376,59
30853080700038,59636,50.69481,3.028376,59
30925823400025,59636,50.69481,3.028376,59
31007069300018,59195,50.63102,2.965393,59
30395274100047,59195,50.63102,2.965393,59
…,…,…,…,…
30034580801630,62624,50.413018,2.992273,62
30613890000445,62624,50.412119,2.982439,62
30613890001138,62624,50.41571,2.981033,62


In [23]:
# sauvegarder l'ensemble du dataframe mtn
departements = [62, 59]
reader = pl.read_csv_batched(
    GEOLOC_PATH,
    separator=";",
    ignore_errors=True,
    columns=['siret', 'x_longitude', 'y_latitude', 'plg_code_commune']
    )
batches = reader.next_batches(100)
# trouver le nb de batches qui rend le truc le + rapide 
i = 0
while batches:  
    df_current_batches = pl.concat(batches)
    # process df
    df_current_batches = df_current_batches.with_columns((df_current_batches["plg_code_commune"]//1000).rename("departement"))
    df_current_batches = df_current_batches.filter(pl.col("departement").is_in(departements))

    # save df
    df_current_batches.write_csv(file=f"./data/output/geoloc/geoloc_out_{i}.csv", include_header=True)
    i += 1
    batches = reader.next_batches(100)

# update etablissements

In [24]:
PATH_ETABLISSEMENTS = r".\data\StockEtablissement_utf8\StockEtablissement_utf8.csv"

In [25]:
headers_to_keep = [
        'siren',
        'siret',
        'trancheEffectifsEtablissement',
        'activitePrincipaleEtablissement',
        'codeCommuneEtablissement'
    ]

In [26]:
reader = pl.read_csv_batched(
    PATH_ETABLISSEMENTS,
    separator=",",
    ignore_errors=True,
    columns=headers_to_keep
    )  
batches = reader.next_batches(100)

In [27]:
import csv
naf_codes = []
with open("output/interesting_naf_codes.csv", 'r', encoding='utf-8') as file:
    csv_reader = csv.DictReader(file, delimiter='|')
    for line in csv_reader:
        naf_codes.append(line["Code NAF"][:2] + '.' + line["Code NAF"][2:])

In [28]:
naf_codes

['35.11Z',
 '35.12Z',
 '35.13Z',
 '35.14Z',
 '35.21Z',
 '35.22Z',
 '35.23Z',
 '36.00Z',
 '37.00Z',
 '38.11Z',
 '38.12Z',
 '38.21Z',
 '38.22Z',
 '38.32Z',
 '39.00Z',
 '32.11Z',
 '32.12Z',
 '32.13Z',
 '32.20Z',
 '32.30Z',
 '32.40Z',
 '32.50A',
 '32.50B',
 '32.91Z',
 '32.99Z',
 '33.11Z',
 '33.12Z',
 '33.13Z',
 '33.14Z',
 '33.15Z',
 '33.16Z',
 '33.17Z',
 '33.19Z',
 '33.20A',
 '33.20B',
 '33.20C',
 '33.20D',
 '24.10Z',
 '24.20Z',
 '24.31Z',
 '24.32Z',
 '24.33Z',
 '24.34Z',
 '24.42Z',
 '24.42Z',
 '24.43Z',
 '24.44Z',
 '24.45Z',
 '24.46Z',
 '24.51Z',
 '24.52Z',
 '24.53Z',
 '24.54Z',
 '25.11Z',
 '25.12Z',
 '25.21Z',
 '25.29Z',
 '25.30Z',
 '25.40Z',
 '25.50A',
 '25.50B',
 '25.61Z',
 '25.62A',
 '25.62B',
 '25.71Z',
 '25.72Z',
 '25.73A',
 '25.73B',
 '25.91Z',
 '25.92Z',
 '25.93Z',
 '25.94Z',
 '25.99A',
 '25.99B',
 '21.10Z',
 '21.20Z',
 '28.11Z',
 '28.12Z',
 '28.13Z',
 '28.14Z',
 '28.15Z',
 '28.21Z',
 '28.22Z',
 '28.23Z',
 '28.24Z',
 '28.25Z',
 '28.29A',
 '28.29B',
 '28.30Z',
 '28.41Z',
 '28.49Z',

In [29]:
# trouver le nb de batches qui rend le truc le + rapide
# chargement en mémoire des codes nafs

i = 0
while batches:  
    df_current_batches = pl.concat(batches)
    
    # process df
    df_current_batches = df_current_batches.with_columns((df_current_batches["codeCommuneEtablissement"]//1000).rename("departement"))
    df_current_batches = df_current_batches.filter(pl.col("departement").is_in(departements))
    df_current_batches = df_current_batches.filter(pl.col("trancheEffectifsEtablissement").is_not_null())
    df_current_batches = df_current_batches.filter(pl.col("activitePrincipaleEtablissement").is_in(naf_codes))
    # save df
    df_current_batches.write_csv(file=f"./data/output/etablissement/etablissement_{i}.csv", include_header=True)
    i += 1
    batches = reader.next_batches(100)

In [30]:
# merger et faire une jointure des 2

# lists paths in etablissmeent
DIR_ETABLISSEMENTS = r".\data\output\etablissement"
paths_etablissement = os.listdir(DIR_ETABLISSEMENTS)
list_df = []
for PATH_FILE_ETABLISSEMENT in paths_etablissement:
    PATH_ETABLISSEMENT = os.path.join(DIR_ETABLISSEMENTS, PATH_ETABLISSEMENT)
    list_df.append(pl.read_csv(PATH_ETABLISSEMENT))


In [31]:
df_etablissements = pl.concat(list_df)

In [32]:
# merger et faire une jointure des 2

# lists paths in etablissmeent
DIR_GEOLOC = r".\data\output\geoloc"
paths_geoloc = os.listdir(DIR_GEOLOC)
list_df = []
for PATH_FILE_GEOLOC in paths_geoloc:
    PATH_GEOLOC = os.path.join(DIR_GEOLOC, paths_geoloc[0])
    list_df.append(pl.read_csv(PATH_GEOLOC))
df_geoloc = pl.concat(list_df)

In [33]:
df_etablissements

siren,siret,trancheEffectifsEtablissement,codeCommuneEtablissement,activitePrincipaleEtablissement,departement
i64,i64,i64,i64,str,i64
45550571,4555057100041,11,59178,"""36.00Z""",59
45650454,4565045400023,12,59327,"""37.00Z""",59
45650538,4565053800023,22,59574,"""25.92Z""",59
45750601,4575060100010,11,59178,"""25.61Z""",59
45750817,4575081700012,22,59449,"""10.83Z""",59
…,…,…,…,…,…
345308936,34530893600029,21,59273,"""24.10Z""",59
345310965,34531096500024,3,59202,"""25.11Z""",59
345332977,34533297700064,12,62276,"""28.41Z""",62


In [34]:
df_geoloc

siret,plg_code_commune,y_latitude,x_longitude,departement
i64,i64,f64,f64,i64
30989139800039,59636,50.69481,3.028376,59
30853080700038,59636,50.69481,3.028376,59
30925823400025,59636,50.69481,3.028376,59
31007069300018,59195,50.63102,2.965393,59
30395274100047,59195,50.63102,2.965393,59
…,…,…,…,…
35132065000168,59256,50.578354,3.131458,59
34130073900024,59256,50.578354,3.131458,59
34323414201109,59256,50.57855,3.131582,59


In [35]:
df_total = df_etablissements.join(df_geoloc, on=["siret", "departement"])

In [36]:
df_total = df_total.rename({
    'x_longitude': 'lng',
    'y_latitude': 'lat',
    'trancheEffectifsEtablissement': 'RH',
    'activitePrincipaleEtablissement': 'NAF'
    })

In [37]:
dict_rh = {
    0: 0,
    1: 2,
    2: 4,
    3: 8,
    11: 15,
    12: 35,
    21: 75,
    22: 150,
    31: 225,
    32: 350,
    41: 750,
    42: 1500,
    51: 3500,
    52: 7500,
    53: 10000
}

In [38]:
df_total

siren,siret,RH,codeCommuneEtablissement,NAF,departement,plg_code_commune,lat,lng
i64,i64,i64,i64,str,i64,i64,f64,f64
303467476,30346747600022,11,62331,"""25.62B""",62,62331,50.274984,2.852314
303467476,30346747600022,11,62331,"""25.62B""",62,62331,50.274984,2.852314
303467476,30346747600022,11,62331,"""25.62B""",62,62331,50.274984,2.852314
303467476,30346747600022,11,62331,"""25.62B""",62,62331,50.274984,2.852314
303467476,30346747600022,11,62331,"""25.62B""",62,62331,50.274984,2.852314
…,…,…,…,…,…,…,…,…
344280979,34428097900057,3,59034,"""33.14Z""",59,59034,50.541762,3.098688
344280979,34428097900057,3,59034,"""33.14Z""",59,59034,50.541762,3.098688
344280979,34428097900057,3,59034,"""33.14Z""",59,59034,50.541762,3.098688


In [39]:
df_total = df_total.with_columns(
    pl.col("RH").replace_strict(
        dict_rh
    )
)

In [40]:
df_total 

siren,siret,RH,codeCommuneEtablissement,NAF,departement,plg_code_commune,lat,lng
i64,i64,i64,i64,str,i64,i64,f64,f64
303467476,30346747600022,15,62331,"""25.62B""",62,62331,50.274984,2.852314
303467476,30346747600022,15,62331,"""25.62B""",62,62331,50.274984,2.852314
303467476,30346747600022,15,62331,"""25.62B""",62,62331,50.274984,2.852314
303467476,30346747600022,15,62331,"""25.62B""",62,62331,50.274984,2.852314
303467476,30346747600022,15,62331,"""25.62B""",62,62331,50.274984,2.852314
…,…,…,…,…,…,…,…,…
344280979,34428097900057,8,59034,"""33.14Z""",59,59034,50.541762,3.098688
344280979,34428097900057,8,59034,"""33.14Z""",59,59034,50.541762,3.098688
344280979,34428097900057,8,59034,"""33.14Z""",59,59034,50.541762,3.098688


In [41]:
OUTPUT_PATH = r".\data\output\ouput.csv"
df_total.write_csv(OUTPUT_PATH)